In [1]:
from sklearn.model_selection import train_test_split


from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from catboost import CatBoostClassifier

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import pickle

In [2]:
with open('/Users/robertpagano/src/metis_project_mcnulty/data/final_for_modeling/final_features_catboost_trimmed.pickle', 'rb') as f:
    df_X = pickle.load(f)

In [3]:
df_X.head()

,outcome_type_Transfer,intake_type,intake_condition,sex_fixed_intake,intake_month_name,weekday_intake,intake_temp,intake_precip,real_age_intake_days,cat_or_kitten,breed_type,intakes_last_week
0,1,Stray,Normal,Unknown,October,Tuesday,75.4,0.0,7,Kitten,Domestic Breed,1.0
1,1,Stray,Normal,Unknown,October,Tuesday,75.4,0.0,7,Kitten,Domestic Breed,2.0
2,1,Stray,Normal,Unknown,October,Tuesday,75.4,0.0,7,Kitten,Domestic Breed,3.0
3,1,Owner Surrender,Normal,Neutered Male,October,Tuesday,75.4,0.0,6209,Cat,Domestic Breed,4.0
4,1,Stray,Normal,Intact Female,October,Tuesday,75.4,0.0,21,Kitten,Domestic Breed,5.0


In [4]:
X = df_X.iloc[:, 1:]
y = df_X.iloc[:, 0]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X.dtypes

intake_type              object
intake_condition         object
sex_fixed_intake         object
intake_month_name        object
weekday_intake           object
intake_temp             float64
intake_precip           float64
real_age_intake_days      int64
cat_or_kitten            object
breed_type               object
intakes_last_week       float64
dtype: object

In [19]:
feature_list

['intake_type',
 'intake_condition',
 'sex_fixed_intake',
 'intake_month_name',
 'weekday_intake',
 'intake_temp',
 'intake_precip',
 'real_age_intake_days',
 'cat_or_kitten',
 'breed_type',
 'intakes_last_week']

In [25]:
df_X['intake_precip'].unique()

array([0.  , 0.43, 0.76, 0.03, 5.49, 0.75, 0.42, 0.37, 0.48, 0.01, 0.53,
       0.1 , 0.14, 4.52, 0.04, 0.34, 0.28, 1.27, 0.67, 0.02, 0.47, 0.61,
       0.07, 0.15, 0.2 , 0.26, 0.09, 0.11, 0.05, 0.13, 0.06, 0.25, 0.58,
       0.27, 1.14, 3.27, 0.19, 3.09, 0.55, 0.24, 0.49, 0.31, 5.05, 0.98,
       0.41, 0.08, 0.69, 0.21, 1.28, 1.15, 0.16, 0.17, 1.57, 3.4 , 1.38,
       0.22, 0.91, 0.63, 0.45, 0.79, 1.48, 0.36, 0.33, 0.9 , 1.22, 1.49,
       0.57, 0.64, 3.85, 0.29, 0.4 , 0.23, 1.13, 1.63, 0.97, 0.35, 1.97,
       5.2 , 0.87, 1.12, 1.05, 1.86, 2.13, 1.09, 0.5 , 0.92, 2.68, 2.8 ,
       0.44, 1.03, 0.81, 1.52, 1.33, 1.45, 1.25, 0.39, 0.18, 1.59, 1.96,
       0.59, 0.54, 1.1 , 0.6 , 0.77, 0.72, 2.12, 0.85, 1.07, 1.06, 1.42,
       1.51, 1.81, 0.12, 1.8 , 0.62, 0.94, 1.76, 0.3 , 1.19, 0.73, 1.31,
       0.7 , 0.32, 0.66, 0.56, 0.68, 1.39, 1.7 , 0.71, 7.  , 1.6 , 1.64,
       0.38, 1.32, 2.72, 0.8 , 3.68, 1.3 , 1.77, 1.87, 1.35, 0.96, 1.62,
       2.11, 0.51, 1.08, 2.31, 2.5 , 1.16, 2.21])

In [7]:
from catboost import Pool, cv

In [8]:
categorical_features_indices = np.where(X.dtypes == np.object)[0]


In [9]:
categorical_features_indices

array([0, 1, 2, 3, 4, 8, 9])

In [10]:
## Functionizing my metrics

def auc(m, X_train, X_test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(X_train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(X_test)[:,1]))

def acc(m, X_train, X_test): 
    return (metrics.accuracy_score(y_train,m.predict_proba(X_train)[:,1]),
                            metrics.accuracy_score(y_test,m.predict_proba(X_test)[:,1]))

def f1(m, X_train, X_test): 
    return (metrics.f1_score(y_train,m.predict_proba(X_train)[:,1]),
                            metrics.f1_score(y_test,m.predict_proba(X_test)[:,1]))

def prec(m, X_train, X_test): 
    return (metrics.precision_score(y_train,m.predict_proba(X_train)[:,1]),
                            metrics.precision_score(y_test,m.predict_proba(X_test)[:,1]))

def recall(m, X_train, X_test): 
    return (metrics.recall_score(y_train,m.predict_proba(X_train)[:,1]),
                            metrics.recall_score(y_test,m.predict_proba(X_test)[:,1]))

In [11]:
from sklearn import metrics


In [12]:
# #Without Categorical features
# params = {'depth': [4, 7, 10],
#           'learning_rate' : [0.03, 0.1, 0.15],
#          'l2_leaf_reg': [1,4,9],
#          'iterations': [300]}
# cb = CatBoostClassifier()
# cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
# cb_model.fit(X_train, y_train)

#With Categorical features
clf = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(X_train,y_train, cat_features= categorical_features_indices)
auc(clf, X_train, X_test)



0:	total: 508ms	remaining: 4m 13s
1:	total: 647ms	remaining: 2m 41s
2:	total: 1s	remaining: 2m 46s
3:	total: 1.06s	remaining: 2m 11s
4:	total: 1.22s	remaining: 2m
5:	total: 1.57s	remaining: 2m 9s
6:	total: 1.64s	remaining: 1m 55s
7:	total: 1.98s	remaining: 2m 1s
8:	total: 2.31s	remaining: 2m 6s
9:	total: 2.42s	remaining: 1m 58s
10:	total: 2.6s	remaining: 1m 55s
11:	total: 2.65s	remaining: 1m 47s
12:	total: 2.98s	remaining: 1m 51s
13:	total: 3.07s	remaining: 1m 46s
14:	total: 3.15s	remaining: 1m 42s
15:	total: 3.18s	remaining: 1m 36s
16:	total: 3.49s	remaining: 1m 39s
17:	total: 3.52s	remaining: 1m 34s
18:	total: 3.57s	remaining: 1m 30s
19:	total: 3.71s	remaining: 1m 29s
20:	total: 4.06s	remaining: 1m 32s
21:	total: 4.45s	remaining: 1m 36s
22:	total: 4.49s	remaining: 1m 33s
23:	total: 4.83s	remaining: 1m 35s
24:	total: 4.88s	remaining: 1m 32s
25:	total: 5.23s	remaining: 1m 35s
26:	total: 5.29s	remaining: 1m 32s
27:	total: 5.33s	remaining: 1m 29s
28:	total: 5.37s	remaining: 1m 27s
29:	to

KeyboardInterrupt: 

In [ ]:
clf.feature_importances_

In [16]:
feature_list = list(X.columns)

In [18]:
feature_list

['intake_type',
 'intake_condition',
 'sex_fixed_intake',
 'intake_month_name',
 'weekday_intake',
 'intake_temp',
 'intake_precip',
 'real_age_intake_days',
 'cat_or_kitten',
 'breed_type',
 'intakes_last_week']

In [17]:
importances = list(clf.feature_importances_)

CatboostError: There is no trained model to use `feature_importances_`. Use fit() to train model. Then use `feature_importances_`.

In [ ]:
cb_feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

In [ ]:
cb_feature_importances

In [ ]:
feature_imp_df = pd.DataFrame(data=cb_feature_importances)

In [ ]:
feature_imp_df.rename(columns={0: 'Feature', 1: 'Importance'}, inplace=True)
feature_imp_df

In [ ]:
feature_imp_df['Feature'] = feature_imp_df['Feature'].str.replace('_',' ')

feature_imp_df

In [ ]:
feature_imp_df['Feature'] = feature_imp_df['Feature'].str.title()
feature_imp_df['Feature'] = feature_imp_df['Feature'].str.replace(' Or ',' or ')
feature_imp_df['Feature'] = feature_imp_df['Feature'].str.replace('Real Age Intake Days','Age in Days')
feature_imp_df

In [ ]:
feature_imp_df = feature_imp_df.sort_values(by=['Importance'])
feature_imp_df

In [ ]:
sns.barplot(x='Importance', y='Feature', data=feature_imp_df, palette="ch:2.5,-.2,dark=.3").set_title('CatBoost Feature Importance')

In [ ]:
outcome_totals = sns.countplot(x='outcome_type_Transfer', data=df_X, palette="ch:2.5,-.2,dark=.3").set_title('Cat Outcomes, 2013-now');
plt.xlabel("Outcome Type")
plt.ylabel("Count")
labels = ['Adoption', 'Transfer']
plt.xticks(df_X['outcome_type_Transfer'], labels)



In [ ]:
def make_confusion_matrix(model, threshold=0.5):
   # Predict class 1 if probability of being in class 1 is greater than threshold
   # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    cmap = sns.cubehelix_palette(8, start=2, rot=0, dark=0, light=.95, reverse=False)
    transfer_confusion = metrics.confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(transfer_confusion, cmap=cmap, annot=True, square=True, fmt='d',
        xticklabels=['Adoption', 'Transfer'],
        yticklabels=['Adoption', 'Transfer']);
    plt.xlabel('Prediction')
    plt.ylabel('Actual')

In [ ]:
make_confusion_matrix(clf, threshold=0.5)

In [ ]:
make_confusion_matrix(clf, threshold=0.6)

In [ ]:
# f1_catboost_test = f1(clf, X_train, X_test)
# accuracy_catboost_test = acc(clf, X_train, X_test)
# precision_catboost_test = prec(clf, X_train, X_test)
# recall_catboost_test = recall(clf, X_train, X_test)
AUC_catboost_test = auc(clf, X_train, X_test)



In [ ]:
AUC_catboost_test

In [ ]:
f1_clf_test = f1_score(y_test, clf.predict(X_test))
accuracy_clf_test = accuracy_score(y_test, clf.predict(X_test))
precision_clf_test = precision_score(y_test, clf.predict(X_test))
recall_clf_test = recall_score(y_test, clf.predict(X_test))
AUC_clf_test = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])

print(f'F1 test score is {f1_clf_test}')
print(f'Accuracy test score is {accuracy_clf_test}')
print(f'precision test score is {precision_clf_test}')
print(f'recall test score is {recall_clf_test}')
print(f'AUC test score is {AUC_clf_test}')

In [ ]:
clf_withplot = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9)
clf_withplot.fit(X_train, y_train, cat_features= categorical_features_indices, plot=True)
auc(clf, X_train, X_test)

Below attempting to plot confusion matrix

In [ ]:
y_pred = clf.predict(X_test)
type(y_pred)

In [ ]:
conf_matrix = metrics.confusion_matrix(y_test, y_pred)
conf_matrix

In [ ]:
with open('/Users/robertpagano/src/metis_project_mcnulty/final_model/catboost_model_v1.pkl', 'wb') as to_write:
    pickle.dump(clf, to_write)

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = clf.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('ROC Curve for CatBoost')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# # method II: ggplot
# from ggplot import *
# df = pd.DataFrame(dict(fpr = fpr, tpr = tpr))
# ggplot(df, aes(x = 'fpr', y = 'tpr')) + geom_line() + geom_abline(linetype = 'dashed')

In [ ]:
sns.countplot(x='')